# ISOT Streamflow isotopes dataset extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is used to retrieve and concatenate the stream water isotopes dataset provided by ISOT.

The output is one file per catchemnt (similar to the CAMELS-CH), with 4 columns:

* date_start
* date_end
* delta_2h
* delta_18o

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas=2.1.3
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* ISOT Flussdaten Messperiode für CAMELS_CH.xlsx


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 


## References
* ISOT. ISOT module of the NAQUA National Groundwater Monitoring. https://www.bafu.admin.ch/bafu/en/home/topics/water/info-specialists/state-of-waterbodies/state-of-groundwater/naqua-national-groundwater-monitoring/isot-module.html (last access 20 Sep 2024).
## Observations
* None

# Import modules

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import tqdm as tqdm
import os
import glob
import warnings
import re

# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = ".."

# Suppress all warnings
warnings.filterwarnings("ignore")

# Path to where the data are stored
path_isot = r"C:\Users\nascimth\Documents\data\CAMELS_CH_Chem\\"

* #### The users should NOT change anything in the code below here. 

In [3]:
# Non-editable variables:
PATH_OUTPUT = r"results\water_isotopes\isot\\"

# Set the directory:
os.chdir(PATH)

# Import data

- Network

In [4]:
# Network CAMELS_CH_Chem
network_camels_ch_chem = pd.read_excel(path_isot+"data/CAMELS_CH_chem_stations_short_v2.xlsx", sheet_name='all_4')
#network_camels_ch_chem.set_index("basin_id", inplace=True)
network_camels_ch_chem

basin_id  bafu_id  naduf_id  nawa_id isot_id            hydro_station  \
0        2009   2009.0    1837.0   1837.0   NIO04            Porte du Scex   
1        2011   2011.0       NaN   4070.0     NaN                     Sion   
2        2016   2016.0    1833.0   1833.0   NIO02                    Brugg   
3        2018   2018.0    1835.0   1339.0     NaN                Mellingen   
4        2019   2019.0       NaN   1852.0   NIO01              Brienzwiler   
..        ...      ...       ...      ...     ...                      ...   
111      2617   2617.0       NaN      NaN     NaN                  Müstair   
112      2623   2623.0       NaN      NaN     NaN                 Oberwald   
113      2634   2634.0    6169.0   1181.0     NaN                    Emmen   
114      2635   2635.0       NaN      NaN     NaN        Einsiedeln, Gross   
115      2640      NaN       NaN   1504.0     NaN  Delémont, Pré-Guillaume   

    hydrowater_body     lon     lat  area_camels  ...  lon_naduf lat_naduf  \
0             Rhône  557660  133280       5239.4  ...   557660.0  133280.0   
1             Rhône  593770  118630       3372.4  ...        NaN       NaN   
2              Aare  657000  259360      11681.3  ...   657000.0  259360.0   
3             Reuss  662830  252580       3385.8  ...   662830.0  252580.0   
4              Aare  649930  177380        555.2  ...        NaN       NaN   
..              ...     ...     ...          ...  ...        ...       ...   
111             Rom  830800  168700        128.6  ...        NaN       NaN   
112           Rhone  669900  154075         93.3  ...        NaN       NaN   
113     Kleine Emme  663700  213630        478.3  ...   663700.0  213630.0   
114       Grossbach  700710  218125          8.9  ...        NaN       NaN   
115           Sorne  593380  245940        213.9  ...        NaN       NaN   

    area_naduf bafu_naduf_distance        nawa_station  lon_nawa  lat_nawa  \
0       5239.4                 0.0       Porte du Scex  557660.0  133280.0   
1          NaN                 NaN                Sion  593277.0  118449.0   
2      11681.3                 0.0               Brugg  657000.0  259360.0   
3       3385.8                 0.0          Gebenstorf  659450.0  258850.0   
4          NaN                 NaN  Brienzerseeeinlauf  646692.0  177000.0   
..         ...                 ...                 ...       ...       ...   
111        NaN                 NaN                 NaN       NaN       NaN   
112        NaN                 NaN                 NaN       NaN       NaN   
113      478.3                 0.0        Emmen-Littau  663917.0  213356.0   
114        NaN                 NaN                 NaN       NaN       NaN   
115        NaN                 NaN            Delémont  593800.0  246600.0   

        area_nawa bafu_nawa_distance remarks.1  
0     5239.402096                0.0       NaN  
1     3372.417040                0.0       NaN  
2    11681.282882                0.0       NaN  
3     3420.503458               10.0       NaN  
4      555.808970                3.3       NaN  
..            ...                ...       ...  
111           NaN                NaN       NaN  
112           NaN                NaN       NaN  
113    478.277165                0.6       NaN  
114           NaN                NaN       NaN  
115    213.930000                0.7       NaN  

[116 rows x 31 columns]

In [ ]:
for sheet_id in range(9):

    # Load the Excel file, selecting the second sheet (index 1) and skipping rows
    dataset_isot = pd.read_excel(
        path_isot+"data/ISOTOPES/ISOT Flussdaten Messperiode für CAMELS_CH.xlsx",
        sheet_name=sheet_id,  # Index for the second sheet (0 is the first)
        skiprows=12  # Skips the first 11 rows, adjust according to your needs
    )

    name_isot = pd.read_excel(
        path_isot+"data/ISOTOPES/ISOT Flussdaten Messperiode für CAMELS_CH.xlsx",
        sheet_name=sheet_id,  # Index for the second sheet (0 is the first)
    )

    name_isot = name_isot.iloc[3, 1]

    code_isot = pd.read_excel(
        path_isot+"data/ISOTOPES/ISOT Flussdaten Messperiode für CAMELS_CH.xlsx",
        sheet_name=sheet_id,  # Index for the second sheet (0 is the first)
    )

    code_isot = code_isot.iloc[4, 1]

    dataset_isot.columns = ["date_start", "date_end", "delta_2h", "delta_18o"]
    dataset_isot["date_start"] = pd.to_datetime(dataset_isot["date_start"], format='%Y-%m-%d')
    dataset_isot["date_end"] = pd.to_datetime(dataset_isot["date_end"], format='%Y-%m-%d')
    dataset_isot.set_index("date_start", inplace=True)
    basin_id_name = str(network_camels_ch_chem[network_camels_ch_chem.isot_id == code_isot].loc[:, "basin_id"].values[0])

    dataset_isot.to_csv(PATH_OUTPUT + "/camels_ch_chem_isot_"+str(basin_id_name)+".csv", encoding='latin')
    
    print(name_isot, code_isot)

Rhein - Diepoldsau NIO03
Rhein - Weil NIO08
Aare - Brienzwiler NIO01
Aare - Thun NIO07
Aare - Brugg NIO02
Rhône - Porte du Scex NIO04
Rhône - Chancy NIO09
Ticino - Riazzino NIO05
Inn - S-chanf NIO06


Observations
- We have 9 stations in total

# End